$\LARGE{African\hspace{0.1cm}Institute\hspace{0.1cm}For\hspace{0.1cm}Mathematical\hspace{0.1cm}Sciences,\hspace{0.1cm}AIMS-Sénégal\\}$

$\Large{Cours\hspace{0.1cm}de: Deep\hspace{0.1cm}Learning\hspace{0.1cm}and\hspace{0.1cm}Neural\hspace{0.1cm}Network\\}$

Responsable du cours: $\Large{\textbf{Prof. Stephane KOUAMO}}$

$\large{\textbf{Projet sur la prédiction des valeurs manquantes du fichier portant sur l'activité agricole au CAMEROUN en appliquant  une structure de réseau de neurone.}}$

Projet présenté par: $\Large{\textbf{Mikhaël Presley KIBINDA-MOUKENGUE}}$

In [371]:
import pandas as pd
import numpy as np
import time
from keras.layers import Dropout, Dense, Activation
from keras.models import Sequential

In [372]:
from keras.utils import to_categorical

L'objectif principal de ce travail, c'est de faire une prédiction sur des valeurs manquantes du dataset 'don_RN.csv' qui est un fichier de l'activité agricole au CAMEROUN. Le travail consistera de diviser le dataset en deux, en fonction de chaque variable 'Result_collect', 'Dest_principal', 'implicaion_femme' de la manière suivante: un dataset comprenant les valeurs manquantes et l'autre ne comprenant pas celles ci pour chaque variable respectives. Nous nous focaliserons sur les datasets n'ayant pas des valeurs manquantes, on va l'entrainer par une structure de réseau de neurone pour obtenir la première prédiction sur une colonne (target, ici on va commencer avec 'Result_collect'). Les valeurs prédictes ainsi trouvées, on va les remplacer dans le dataset ayant des valeurs manquantes(en ce qui concerne uniquement la colonne 'Result_collect'). Ainsi, nous allons reconstruire un nouveau dataset qui n'aura que deux colonnes cette fois des valeurs manquantes, faire le même travail expliqué precedemment. On va constater dans notre réseau de neurone que les entrées de la couche $i$ sont les sorties de la $\hspace{0.1cm}i+1$.

## Affichage du dataset

In [373]:
df=pd.read_csv('don_RN.csv')

In [374]:
df.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,1.0,2.0,NaN,NaN,4.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,2.0,NaN,NaN,4.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,3.0,2.0,NaN,NaN,4.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,4.0,1.0,2.0,1.0,4.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,5.0,1.0,1.0,1.0,4.0


### Vérification des données manquantes

In [375]:
df.isna().sum()

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Result_collect         240
Milieu_Red               0
Code_spe                 0
Prodution_pecul          1
Dest_principal      201331
implicaion_femme    201332
code_agent               0
dtype: int64

#### Vérification de la ligne de "Prodution_pecul" qui a une valeur manquante

In [376]:
loc=df[df[['Prodution_pecul']].isnull().any(axis=1)]

In [377]:
loc

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent
97314,4.0,21.0,139.0,2.0,2.0,14.9396,12.09216,8.0,2.0,10.0,NaN,NaN,NaN,55.0


#### Suppréssion de cette ligne

In [378]:
df.drop(97314,0,inplace=True)

In [379]:
df.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,1.0,2.0,NaN,NaN,4.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,2.0,NaN,NaN,4.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,3.0,2.0,NaN,NaN,4.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,4.0,1.0,2.0,1.0,4.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,5.0,1.0,1.0,1.0,4.0


### Copie de notre dataset

In [380]:
dff=df.copy

## Statistique des données

#### Taille ou dimension du dataset

In [381]:
df.shape

(260019, 14)

In [382]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Region,260019.0,4.988832,2.567918,1.0,3.00000,4.00000,7.00000,10.0
Depart,260019.0,27.062153,14.472168,1.0,18.00000,24.00000,38.00000,58.0
Arond,260019.0,173.994289,93.818561,1.0,110.00000,163.00000,246.00000,360.0
Ville,260019.0,3.443818,8.964635,0.0,2.00000,2.00000,4.00000,999.0
Village_quart,260019.0,23.640130,36.077804,0.0,5.00000,12.00000,27.00000,714.0
Longitude,260019.0,10.086845,6.992207,0.0,5.47731,10.53494,13.55092,522.0
Latitude,260019.0,9.582849,6.579964,0.0,5.60172,10.13752,12.01432,522.0
Result_collect,259779.0,1.002610,0.133851,1.0,1.00000,1.00000,1.00000,8.0
Milieu_Red,260019.0,1.828205,0.377203,1.0,2.00000,2.00000,2.00000,2.0
Code_spe,260019.0,7.999896,4.320490,1.0,4.00000,8.00000,12.00000,15.0


In [383]:
df.corr()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent
Region,1.000000,0.990170,0.985954,-0.020720,-0.073676,0.017769,-0.163146,-0.013577,0.034349,0.000009,0.003810,-0.012002,0.048506,0.783973
Depart,0.990170,1.000000,0.993847,-0.014511,-0.074208,0.010176,-0.152093,-0.014871,0.025881,0.000010,0.007126,0.000548,0.050006,0.772670
Arond,0.985954,0.993847,1.000000,-0.013320,-0.055336,0.035873,-0.142246,-0.014931,0.027155,0.000009,0.003162,-0.015263,0.046814,0.780323
Ville,-0.020720,-0.014511,-0.013320,1.000000,0.100788,0.040255,-0.013141,-0.004490,0.071135,0.000004,0.002339,-0.011202,0.001690,-0.000649
Village_quart,-0.073676,-0.074208,-0.055336,0.100788,1.000000,0.131085,0.005399,-0.006967,0.090277,0.000014,-0.014612,-0.066314,-0.036360,-0.059986
Longitude,0.017769,0.010176,0.035873,0.040255,0.131085,1.000000,0.555702,0.001334,-0.007844,-0.000017,-0.028029,-0.045950,-0.004542,0.022206
Latitude,-0.163146,-0.152093,-0.142246,-0.013141,0.005399,0.555702,1.000000,-0.007862,-0.000233,-0.000009,-0.027553,0.031490,-0.033552,-0.103368
Result_collect,-0.013577,-0.014871,-0.014931,-0.004490,-0.006967,0.001334,-0.007862,1.000000,-0.007135,-0.001258,0.000209,-0.011769,-0.006206,-0.012354
Milieu_Red,0.034349,0.025881,0.027155,0.071135,0.090277,-0.007844,-0.000233,-0.007135,1.000000,-0.000011,-0.009816,0.058720,-0.031810,0.006947
Code_spe,0.000009,0.000010,0.000009,0.000004,0.000014,-0.000017,-0.000009,-0.001258,-0.000011,1.000000,0.222866,0.083338,-0.076269,-0.000018


In [384]:
df.isna().sum() #Vérification à nouveau, des valeurs manquantes

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Result_collect         240
Milieu_Red               0
Code_spe                 0
Prodution_pecul          0
Dest_principal      201330
implicaion_femme    201331
code_agent               0
dtype: int64

## Suppréssion des colonnes ['Dest_principal','implicaion_femme']

In [385]:
data=df.drop(columns=['Dest_principal','implicaion_femme'])
data.isna().sum()  # Vérification des valeurs manquantes

Region               0
Depart               0
Arond                0
Ville                0
Village_quart        0
Longitude            0
Latitude             0
Result_collect     240
Milieu_Red           0
Code_spe             0
Prodution_pecul      0
code_agent           0
dtype: int64

## Affichage du dataset dont la colonne 'Result_collect' contient uniquement les valeurs manquantes

In [386]:
X_1=data[data['Result_collect'].isnull()]

In [387]:
X_1

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Result_collect,Milieu_Red,Code_spe,Prodution_pecul,code_agent
27525,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,1.0,2.0,11.0
27526,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,2.0,2.0,11.0
27527,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,3.0,2.0,11.0
27528,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,4.0,2.0,11.0
27529,2.0,8.0,41.0,2.0,1.0,11.06457,4.408060,NaN,2.0,5.0,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
257615,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,11.0,2.0,73.0
257616,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,12.0,2.0,73.0
257617,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,13.0,2.0,73.0
257618,10.0,58.0,352.0,3.0,3.0,9.04432,4.290573,NaN,2.0,14.0,2.0,73.0


# First model

## Affichage du dataset ayant aucune valeur manquante

In [388]:
data_1=data.dropna(subset=['Result_collect'])

#### Dimension

In [389]:
data_1.shape

(259779, 12)

#### Vérification des valeurs manquantes

In [390]:
data_1.isna().sum()

Region             0
Depart             0
Arond              0
Ville              0
Village_quart      0
Longitude          0
Latitude           0
Result_collect     0
Milieu_Red         0
Code_spe           0
Prodution_pecul    0
code_agent         0
dtype: int64

# Echantillonage, Séparation en Features et Target

### Features

In [391]:
X=data_1.drop(columns=['Result_collect']).astype('int32')

### Target

In [392]:
y=data_1[['Result_collect']].astype('int32')

# Splitting de notre échantillon en train et test

In [393]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

### Dimensions des training et testing sets

In [394]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((207823, 11), (51956, 11), (207823, 1), (51956, 1))

# Structure du Réseau de Neurone

In [395]:
model = Sequential()
model.add(Dense(29, kernel_initializer="uniform", input_dim=11, activation='relu'))
model.add(Dropout(rate = 0.000001))
model.add(Dense(19, kernel_initializer="uniform", activation='relu'))
model.add(Dense(1, kernel_initializer="uniform", activation='softmax'))

In [396]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Entrainement du modèle

In [397]:
start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=256, verbose = 1)
temps_exec = time.time() - start_time
print("\nTemps passé à entraîner le modèle pour 5 epochs : %.3f secondes" % temps_exec)

Epoch 1/5
207823/207823 [==============================] - 1s 6us/step - loss: -0.0333 - accuracy: 0.9997
Epoch 2/5
207823/207823 [==============================] - 1s 6us/step - loss: -0.0333 - accuracy: 0.9997
Epoch 3/5
207823/207823 [==============================] - 1s 6us/step - loss: -0.0333 - accuracy: 0.9997
Epoch 4/5
207823/207823 [==============================] - 1s 5us/step - loss: -0.0333 - accuracy: 0.9997
Epoch 5/5
207823/207823 [==============================] - 2s 7us/step - loss: -0.0333 - accuracy: 0.9997

Temps passé à entraîner le modèle pour 5 epochs : 6.723 secondes


### Prédiction

In [398]:
y_pred=pd.DataFrame(model.predict(X_test))

## Vérification de la performance du modèle

In [399]:
scores = model.evaluate(X_test, y_test, batch_size = 256, verbose = 1)
print("Le taux de réussite est de : %.2f%%" % (scores[1] * 100))

51956/51956 [==============================] - 0s 4us/step
Le taux de réussite est de : 99.94%


# Second model

### Remplacement des valeurs manquantes de la colonne 'Result_collect' par ses valeurs predictes

In [400]:
df[['Result_collect']].fillna(y_pred.mode())

,Result_collect
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
260015,1.0
260016,1.0
260017,1.0
260018,1.0


### Suppréssion de la colonne 'Result_collect' dans le dataset 'df' du depart

In [401]:
df_11=df.drop(columns=['Result_collect'])

### Concaténation de la colonne 'Result_collect' sans valeurs manquantes avec 'df_11' puis création d'un nouveau dataset 'df_1'

In [402]:
df_1=pd.concat([df_11, df[['Result_collect']].fillna(1)],axis = 1)

In [403]:
df_1.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Milieu_Red,Code_spe,Prodution_pecul,Dest_principal,implicaion_femme,code_agent,Result_collect
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,NaN,NaN,4.0,1.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,2.0,2.0,NaN,NaN,4.0,1.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,3.0,2.0,NaN,NaN,4.0,1.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,4.0,1.0,2.0,1.0,4.0,1.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,5.0,1.0,1.0,1.0,4.0,1.0


## Vérification des valeurs manquantes dans 'df_1'

In [404]:
df_1.isnull().sum()

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Milieu_Red               0
Code_spe                 0
Prodution_pecul          0
Dest_principal      201330
implicaion_femme    201331
code_agent               0
Result_collect           0
dtype: int64

### Dimension de 'df_1'

In [405]:
df_1.shape

(260019, 14)

### Copie de 'df_1'

In [406]:
df_f=df_1.copy

### Suppréssion de la colonne 'implicaion_femme'

In [407]:
dataa=df_1.drop(columns=['implicaion_femme'])

## Affichage du dataset dont la colonne 'Dest_principal' contient uniquement les valeurs manquantes

In [408]:
X_2=dataa[dataa['Dest_principal'].isnull()]

## Affichage du dataset ayant aucune valeur manquante

In [409]:
data_2=dataa.dropna(subset=['Dest_principal'])

#### Dimension

In [410]:
data_2.shape

(58689, 13)

#### Vérification des valeurs manquantes

In [411]:
data_2.isna().sum()

Region             0
Depart             0
Arond              0
Ville              0
Village_quart      0
Longitude          0
Latitude           0
Milieu_Red         0
Code_spe           0
Prodution_pecul    0
Dest_principal     0
code_agent         0
Result_collect     0
dtype: int64

## Echantillonage, Séparation en Features et en target

### Features

In [412]:
X=data_2.drop(columns=['Dest_principal']).astype('int32')

### Target

In [413]:
y=data_2[['Dest_principal']].astype('int32')

## Splitting de l'echantillon, en train et test sets

In [414]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#### Dimension de nos training et testing sets

In [415]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((46951, 12), (11738, 12), (46951, 1), (11738, 1))

## Structure du reseau de neurone

In [416]:
model = Sequential()
model.add(Dense(29, kernel_initializer="uniform", input_dim=12, activation='relu'))
model.add(Dropout(rate = 0.000001))
model.add(Dense(19, kernel_initializer="uniform", activation='relu'))
model.add(Dense(1, kernel_initializer="uniform", activation='softmax'))

In [417]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Entrainement du modèle

In [418]:
start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=256, verbose = 1)
temps_exec = time.time() - start_time
print("\nTemps passé à entraîner le modèle pour 5 epochs : %.3f secondes" % temps_exec)

Epoch 1/5
46951/46951 [==============================] - 1s 14us/step - loss: -17.7356 - accuracy: 0.6057
Epoch 2/5
46951/46951 [==============================] - 0s 9us/step - loss: -17.7356 - accuracy: 0.6057
Epoch 3/5
46951/46951 [==============================] - 0s 9us/step - loss: -17.7356 - accuracy: 0.6057
Epoch 4/5
46951/46951 [==============================] - 0s 9us/step - loss: -17.7356 - accuracy: 0.6057
Epoch 5/5
46951/46951 [==============================] - 0s 7us/step - loss: -17.7356 - accuracy: 0.6057

Temps passé à entraîner le modèle pour 5 epochs : 3.014 secondes


### Prédiction

In [419]:
y_pred=pd.DataFrame(model.predict(X_test))

## Performance du modèle

In [420]:
scores = model.evaluate(X_test, y_test, batch_size = 256, verbose = 1)
print("Le taux de réussite est de : %.2f%%" % (scores[1] * 100))

11738/11738 [==============================] - 0s 4us/step
Le taux de réussite est de : 60.93%


# Third model

### Remplacement des valeurs manquantes de la colonne 'Dest_principal' par ses valeurs predictes

In [421]:
y_pred.mode()

,0
0,1.0


In [422]:
df_1[['Dest_principal']].fillna(1)

,Dest_principal
0,1.0
1,1.0
2,1.0
3,2.0
4,1.0
...,...
260015,1.0
260016,1.0
260017,1.0
260018,4.0


### Suppréssion de la colonne 'Dest_principal' dans le dataset 'df_1'

In [423]:
df_111=df_1.drop(columns=['Dest_principal'])

### Concaténation de la colonne 'Dest_principal' sans valeurs manquantes avec 'df_111' puis création d'un nouveau dataset 'df_2'

In [424]:
df_2=pd.concat([df_111, df_1[['Dest_principal']].fillna(1)],axis = 1)

In [425]:
df_2.head()

,Region,Depart,Arond,Ville,Village_quart,Longitude,Latitude,Milieu_Red,Code_spe,Prodution_pecul,implicaion_femme,code_agent,Result_collect,Dest_principal
0,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,1.0,2.0,NaN,4.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,2.0,2.0,NaN,4.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,3.0,2.0,NaN,4.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,4.0,1.0,1.0,4.0,1.0,2.0
4,1.0,1.0,1.0,1.0,1.0,13.27014,6.46195,1.0,5.0,1.0,1.0,4.0,1.0,1.0


### Vérification des valeurs manquantes dans 'df2'

In [426]:
df_2.isnull().sum()

Region                   0
Depart                   0
Arond                    0
Ville                    0
Village_quart            0
Longitude                0
Latitude                 0
Milieu_Red               0
Code_spe                 0
Prodution_pecul          0
implicaion_femme    201331
code_agent               0
Result_collect           0
Dest_principal           0
dtype: int64

### Copie de df_2

In [427]:
df_ff=df_2.copy

## Affichage du dataset dont la colonne 'implicaion_femme' contient uniquement les valeurs manquantes

In [428]:
X_3=df_2[df_2['implicaion_femme'].isnull()]

## Affichage du dataset ayant aucune valeur manquante

In [429]:
data_3=df_2.dropna(subset=['implicaion_femme']) 

## Echantillonage, Séparation en Features et en target

### Features

In [430]:
X=data_3.drop(columns=['implicaion_femme']).astype('int32')

### Target

In [431]:
y=data_3[['implicaion_femme']].astype('int32')

## Splitting de l'echantillon, en train et test sets

In [432]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#### Dimension de nos training et testing sets

In [433]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((46950, 13), (11738, 13), (46950, 1), (11738, 1))

## Structure du réseau de neurone

In [434]:
model = Sequential()
model.add(Dense(29, kernel_initializer="uniform", input_dim=13, activation='relu'))
model.add(Dropout(rate = 0.000001))
model.add(Dense(19, kernel_initializer="uniform", activation='relu'))
model.add(Dense(1, kernel_initializer="uniform", activation='softmax'))

In [435]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Entrainement du modèle

In [436]:
start_time = time.time()
model.fit(X_train, y_train, epochs=5, batch_size=256, verbose = 1)
temps_exec = time.time() - start_time
print("\nTemps passé à entraîner le modèle pour 5 epochs : %.3f secondes" % temps_exec)

Epoch 1/5
46950/46950 [==============================] - 0s 8us/step - loss: -2.1264 - accuracy: 0.4704
Epoch 2/5
46950/46950 [==============================] - 0s 6us/step - loss: -2.1264 - accuracy: 0.4704
Epoch 3/5
46950/46950 [==============================] - 0s 6us/step - loss: -2.1264 - accuracy: 0.4704
Epoch 4/5
46950/46950 [==============================] - 0s 8us/step - loss: -2.1264 - accuracy: 0.4704
Epoch 5/5
46950/46950 [==============================] - 0s 6us/step - loss: -2.1264 - accuracy: 0.4704

Temps passé à entraîner le modèle pour 5 epochs : 2.145 secondes


### Prédiction

In [437]:
y_pred=pd.DataFrame(model.predict(X_test))

## Performance du modèle

In [438]:
scores = model.evaluate(X_test, y_test, batch_size = 256, verbose = 1)
print("Le taux de réussite est de : %.2f%%" % (scores[1] * 100))

11738/11738 [==============================] - 0s 5us/step
Le taux de réussite est de : 46.86%


Comme dit précedemment(au debut), nous avons entrainé ce modèle en trois différentes étapes. La première consistait à prédire en utilisant un réseau de neurones les valeurs manquantes de la colonne 'Result_collect' ce qui a donné une performance de $0.9994$, la seconde consistait à prédire les valeurs manquantes de la colonne 'Dest_principal' et le modèle a donné une performance de $0.6093$ et la troisème et dernière étape consistait à prédire les valeurs manquantes de la colonne 'implicaion_femme' avec une performance du modèle de $0.4686$. On remarque que cette manière de faire n'est pas bonne, car de plus en plus notre modèle devient moins bon, la performance du modèle décroît considérablement. On pouvait aussi bien appliquer un seul modèle de réseau de neurone pour ce genre de problème, ce qui pourrait améliorer ce genre de travail. 

$$\Huge{\textbf{FIN}}$$